In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.io import wavfile
from scipy.signal import bessel, freqz, lfilter, butter
import pandas as pd

sample_rate = 44100.

In [ ]:
# X-over filter design
xover_freq = 100.
filter_order = 4

xover_w = xover_freq/sample_rate*2
b_lo, a_lo = bessel(filter_order, xover_w, btype="low")
b_hi, a_hi = bessel(filter_order, xover_w, btype="high")
w, h_lo = freqz(b_lo, a_lo, worN=1024)
w, h_hi = freqz(b_hi, a_hi, worN=1024)
_ = plt.semilogx(w/np.pi * sample_rate/2, 20.*np.log10(np.abs(h_lo)),
        w/np.pi * sample_rate/2, 20.*np.log10(np.abs(h_hi)))

In [ ]:
# Waveshaper design: cheby
x = np.linspace(-1, 1, 101)
cheby5 = lambda x: 16 * x ** 5 - 20 * x ** 3 + 5 * x
y = cheby5(x)
plt.plot(x, y)


In [ ]:
# Test on chess_box
fs, chess_box_in = wavfile.read("chess_box.wav")

assert fs == sample_rate

def test(input, fn, test_case_name):
    input_lpf = lfilter(b_lo, a_lo, input)
    out = fn(input_lpf)
    out *=  1. / np.max(out)
    wavfile.write("chess_box_%s.wav" % test_case_name,
            int(sample_rate), out)
    return out

test(chess_box_in, cheby5, "cheby")

In [ ]:
# Waveshaper design: remainder from bit-crushing
bits = 12
remainder_gain = 10.

chunk_size = 2. ** (-(bits - 1))

b_smooth, a_smooth = butter(1, 4000./sample_rate*2)

def bitcrush(input):
    global chunk_size
    global remainder_gain
    input_crushed = np.round(input / chunk_size) * chunk_size
    input_remainder = input - input_crushed
    input_grunged = input_crushed + input_remainder * remainder_gain
    return lfilter(b_smooth, a_smooth, input_grunged)

x = np.linspace(-1, 1, 101)
y = bitcrush(x)
plt.plot(x, y)

In [ ]:
# Test bitcrushing on chessbox

test(chess_box_in, bitcrush, "bitcrush")

In [ ]:
# White noise with envelope

# Design A/R smoothing filter

responsiveness = 0.01

b_envsm, a_envsm = butter(1, responsiveness)

def whitenoiseenv(input):
    noise = np.random.random(input.shape) * 2 - 1
    input_envelope = np.abs(input)
    input_envelope_lpf = lfilter(b_envsm, a_envsm, input_envelope)
    return noise * input_envelope

test(chess_box_in, whitenoiseenv, "whitenoiseenv")

In [ ]:
# Filtered white noise with envelope

noise_fcut = 1000.

b_noise, a_noise = butter(1, noise_fcut / sample_rate * 2)

def whitenoiseenvfilt(input):
    noise = np.random.random(input.shape) * 2 - 1
    input_envelope = np.abs(input)
    input_envelope_lpf = lfilter(b_envsm, a_envsm, input_envelope)
    filt_noise = lfilter(b_noise, a_noise, noise)
    return filt_noise * input_envelope

test(chess_box_in, whitenoiseenvfilt, "whitenoiseenvfilt")

In [ ]:
# Voss pink noise

def voss(nrows, ncols=16):
    """Generates pink noise using the Voss-McCartney algorithm.
    
    nrows: number of values to generate
    rcols: number of random sources to add
    
    returns: NumPy array
    """
    array = np.empty((nrows, ncols))
    array.fill(np.nan)
    array[0, :] = np.random.random(ncols)
    array[:, 0] = np.random.random(nrows)
    
    # the total number of changes is nrows
    n = nrows
    cols = np.random.geometric(0.5, n)
    cols[cols >= ncols] = 0
    rows = np.random.randint(nrows, size=n)
    array[rows, cols] = np.random.random(n)

    df = pd.DataFrame(array)
    df.fillna(method='ffill', axis=0, inplace=True)
    total = df.sum(axis=1)

    return total.values


def pinknoiseenv(input):
    noise = voss(len(input), ncols=1)
    input_envelope = np.abs(input)
    input_envelope_lpf = lfilter(b_envsm, a_envsm, input_envelope)
    return noise * input_envelope

test(chess_box_in, pinknoiseenv, "pinknoiseenv")